In [6]:
#inporting the required libraries
import numpy as np
import cv2 as cv

In [7]:
def Make_Mask(hsv, x, y, z):
    # Convert the given BGR color to HSV
    bgr_color = np.uint8([[[x, y, z]]])
    hsv_color = cv.cvtColor(bgr_color, cv.COLOR_BGR2HSV)
    
    # Define the range for HSV values
    low_offset = np.array([7,40,40])
    high_offset = np.array([7, 255, 255]) 

    lower_limit = np.clip(hsv_color[0][0] - low_offset, 0, 255)
    upper_limit = np.clip(hsv_color[0][0] + high_offset, 0, 255)

    # Threshold the HSV image to get only the required colors
    mask = cv.inRange(hsv, lower_limit, upper_limit)

    return mask

In [8]:
def Countour_Coordinate(contours, frame):
    x_avg = 0
    y_avg = 0 
    font = cv.FONT_HERSHEY_COMPLEX
    k = 0
    
    for cnt in contours:
        # Approximate the contour to a polygon
        approx = cv.approxPolyDP(cnt, 0.009 * cv.arcLength(cnt, True), True)
        
        # Draw the contours on the frame
        cv.drawContours(frame, [approx], 0, (0, 0, 255), 2) 
        
        # Flatten the array containing the coordinates of the vertices
        n = approx.ravel()
        
        for i in range(0, len(n), 2):
            x = n[i]
            y = n[i + 1]
            
            # Check if the coordinate is within the specified range
            if 300 < x < 900 and 107 < y < 590:
                k += 1
                x_avg += x
                y_avg += y
    
    # Calculate the average coordinates if any valid points were found
    if k != 0:
        x_avg = x_avg / k
        y_avg = y_avg / k
    
    # Display the average coordinates on the frame
    coord_string = f"{int(x_avg)} {int(y_avg)}"
    cv.putText(frame, coord_string, (int(x_avg), int(y_avg)), font, 0.5, (0, 255, 0))

    return frame

In [9]:
# Open the video file
cap = cv.VideoCapture('tennisVid.mp4')

# Get the frame width and height
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object to save the output video
fourcc = cv.VideoWriter_fourcc('X', 'V', 'I', 'D')
out = cv.VideoWriter("output.mp4", fourcc, 5.0, (1280, 720))

# Read the first frame from the video
ret, frame1 = cap.read()
if ret:
    # Resize the first frame to 1280x720
    frame1 = cv.resize(frame1, (1280, 720), interpolation=cv.INTER_CUBIC)
else:
    print("Failed to read the first frame")
    cap.release()
    exit()

# Read the second frame from the video
ret, frame2 = cap.read()
if ret:
    # Resize the second frame to 1280x720
    frame2 = cv.resize(frame2, (1280, 720), interpolation=cv.INTER_CUBIC)
else:
    print("Failed to read the second frame")
    cap.release()
    exit()

(720, 1280, 3)


In [10]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame is None:
        break
    
    # Calculate the absolute difference between frames
    diff = cv.absdiff(frame1, frame2)
    gray = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv.threshold(blur, 20, 255, cv.THRESH_BINARY)
    dilated = cv.dilate(thresh, None, iterations=3)

    # Create a masked image using the dilated threshold
    res = cv.bitwise_and(frame1, frame1, mask=dilated)

    # Convert to HSV and apply the custom mask function
    hsv = cv.cvtColor(res, cv.COLOR_BGR2HSV)
    mask = Make_Mask(hsv, 255, 255, 200)

    # Find contours and draw them with average coordinates on frame1
    contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    Countour_Coordinate(contours, frame1)

    # Resize frame1 for output and write to the video file
    image = cv.resize(frame1, (1280, 720))
    out.write(image)

    # Optionally display the processed frames
    dilated_resized = cv.resize(dilated, (1280, 720))
    cv.imshow("Dilated", dilated_resized)
    cv.imshow("Feed", frame1)

    # Prepare for the next iteration
    frame1 = frame2
    ret, frame2 = cap.read()
    if ret:
        frame2 = cv.resize(frame2, (1280, 720), interpolation=cv.INTER_CUBIC)
    else:
        break

    # Exit on 'q' key press
    if cv.waitKey(30) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv.destroyAllWindows()